# Azure Key Vault Secret Scope 설정

> **Purpose:** Azure Key Vault와 Databricks Secret Scope를 연동하여
> 소스 코드에 비밀번호/키 값을 노출시키지 않는 보안 환경을 구축합니다.
>
> **참조:** ProjectProposal Problem-Solving #2 (Key Vault IAM 권한 분리)
> - 사용자: Key Vault Administrator (Secret 관리)
> - Databricks: Key Vault Secrets User (읽기 전용, 최소 권한 원칙)

## 1. Secret Scope 설정 가이드

### Databricks CLI 방식 (권장)
```bash
# 1. Secret Scope 생성 (Azure Key Vault 연동)
databricks secrets create-scope \
  --scope key-vault-scope \
  --scope-backend-type AZURE_KEYVAULT \
  --resource-id /subscriptions/{sub-id}/resourceGroups/{rg}/providers/Microsoft.KeyVault/vaults/{vault-name} \
  --dns-name https://{vault-name}.vault.azure.net/

# 2. Secret 목록 확인
databricks secrets list --scope key-vault-scope

# 3. Secret 값 확인 (보안상 마스킹)
databricks secrets list-secrets --scope key-vault-scope
```

## 2. Secret 정의 (Key Vault에 등록 필요)

In [ ]:

# 프로젝트에서 사용하는 Secret 목록
REQUIRED_SECRETS = {
    "openai-api-key": {
        "description": "OpenAI API Key (임상 추론 / Judge 평가)",
        "used_by": ["40_OpenAI_Clinical_Inference.py", "60_LLM_Judge_Pipeline.py", "61_Correction_Summary.py"],
    },
    "storage-account-name": {
        "description": "Azure Storage Account 이름",
        "used_by": ["10_Bronze_Ingestion.py"],
    },
    "storage-account-key": {
        "description": "Azure Storage Account Access Key",
        "used_by": ["10_Bronze_Ingestion.py"],
    },
    "databricks-pat": {
        "description": "Databricks Personal Access Token (SQL API 인증)",
        "used_by": ["ER Dashboard (databricksService.ts)"],
    },
    "gemini-api-key": {
        "description": "Google Gemini API Key (SOAP 노트 생성)",
        "used_by": ["ER Dashboard (geminiService.ts)"],
    },
}

print("📋 필요한 Secret 목록:")
for name, info in REQUIRED_SECRETS.items():
    print(f"  🔑 {name}: {info['description']}")

## 3. Secret 로드 유틸리티

In [ ]:

def get_secret(key_name, scope="key-vault-scope"):
    """
    Azure Key Vault에서 Secret 값을 안전하게 로드합니다.
    
    Args:
        key_name: Secret 이름 (e.g., "openai-api-key")
        scope: Databricks Secret Scope 이름
    
    Returns:
        str: Secret 값
    """
    try:
        value = dbutils.secrets.get(scope=scope, key=key_name)
        print(f"✅ Secret 로드 성공: {key_name} (****)")
        return value
    except Exception as e:
        print(f"⚠️ Secret 로드 실패: {key_name}")
        print(f"   원인: {str(e)}")
        print(f"   → Key Vault에 '{key_name}' Secret이 등록되어 있는지 확인하세요.")
        print(f"   → Databricks에 'Key Vault Secrets User' 역할이 부여되어 있는지 확인하세요.")
        return None

## 4. ADLS Gen2 마운트 설정

In [ ]:

def mount_adls_gen2(container_name, mount_point, storage_account=None):
    """
    ADLS Gen2 컨테이너를 Databricks DBFS에 마운트합니다.
    
    Args:
        container_name: Blob 컨테이너 이름
        mount_point: DBFS 마운트 경로 (e.g., "/mnt/adls-gen2")
        storage_account: Storage Account 이름 (None이면 Key Vault에서 로드)
    """
    if storage_account is None:
        storage_account = get_secret("storage-account-name")
    
    storage_key = get_secret("storage-account-key")
    
    if not storage_account or not storage_key:
        print("⚠️ Storage 인증 정보를 가져올 수 없습니다.")
        return
    
    # 기존 마운트 확인
    existing_mounts = [m.mountPoint for m in dbutils.fs.mounts()]
    if mount_point in existing_mounts:
        print(f"ℹ️ 이미 마운트됨: {mount_point}")
        return
    
    # 마운트 실행
    try:
        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{storage_account}.blob.core.windows.net",
            mount_point=mount_point,
            extra_configs={
                f"fs.azure.account.key.{storage_account}.blob.core.windows.net": storage_key
            }
        )
        print(f"✅ 마운트 성공: {mount_point} → {container_name}")
    except Exception as e:
        print(f"⚠️ 마운트 실패: {str(e)}")

# mount_adls_gen2("raw-telemetry-data", "/mnt/raw-telemetry")
# mount_adls_gen2("bronze", "/mnt/bronze")

## 5. 설정 검증

In [ ]:

def verify_configuration():
    """전체 설정 상태를 검증합니다."""
    print("=" * 60)
    print("🔍 설정 검증")
    print("=" * 60)
    
    # Secret Scope 확인
    try:
        scopes = dbutils.secrets.listScopes()
        print(f"\n📦 Secret Scopes: {[s.name for s in scopes]}")
    except Exception:
        print("\n⚠️ Secret Scope 확인 불가")
    
    # 각 Secret 확인
    print("\n🔑 Secret 상태:")
    for name in REQUIRED_SECRETS:
        try:
            dbutils.secrets.get(scope="key-vault-scope", key=name)
            print(f"  ✅ {name}: 접근 가능")
        except Exception:
            print(f"  ❌ {name}: 접근 불가")
    
    # DBFS 마운트 확인
    print("\n💾 DBFS 마운트:")
    try:
        for m in dbutils.fs.mounts():
            if "/mnt" in m.mountPoint:
                print(f"  📁 {m.mountPoint} → {m.source}")
    except Exception:
        print("  ⚠️ 마운트 정보 확인 불가")
    
    print("\n" + "=" * 60)

# verify_configuration()
print("→ verify_configuration() 호출하여 전체 설정 검증")